<span style="font-size:150%">Merk: denne notebooken er ment som dokumentasjon. Skal live-kodes.</span>

# Setup

In [ ]:
# TMP
%reload_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

In [ ]:
from utils import *

In [ ]:
#%load utils.py

# Data

*Noen ord om CIFAR10: https://www.cs.toronto.edu/~kriz/cifar.html*

*Mål: Bruke NN til å klassifisere disse 32x32x3-bildene i deres 10 ulike klasser (kategorier).*

In [ ]:
trainloader, testloader = get_cifar10_pytorch(batch_size=4)

*Data lastes inn i batches. Best å prosessere data i batches (varians + gpu)*

In [ ]:
trainloader.dataset.train_data.shape

In [ ]:
plot_cifar10_pytorch()

In [ ]:
num_classes

In [ ]:
classes

# Enkelt (gammeldags) nevralt nettverk (i et hypermoderne rammeverk)

<img src="assets/pytorch.png">

*Skyte spurv med kanoner*

In [ ]:
input_size = 32*32*3
hidden_size = 84

In [ ]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()
        
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        x = x.view(x.size(0), -1) # Gjør bildet om til en lang 32*32*3 = 3072-vektor
        x = F.tanh(self.fc1(x)) # Kjør vektoren gjennom et hidden layer og en aktiveringsfunksjon
        x = self.fc2(x) # Prediker klassen ved å kjøre gjennom output-layer
        return x

*fc1 har output* `x = tanh(dot(W, x) + b)`

<img src="assets/tanh.gif">

*Trenger bare definere `forward`-funksjon. `backward` blir automatisk opprettet.*

In [ ]:
net = Net(input_size = input_size, hidden_size = hidden_size, num_classes = num_classes)

In [ ]:
net

In [ ]:
#list(net.parameters())
#len(list(net.parameters()))

## Trening

Husk stegene:
1. Sett opp nevralt nettverk
2. Hent ut batch med treningsdata
3. Kjør disse gjennom nettverket for å få nettverkets prediksjoner
4. Mål avvik mellom fasit og prediskjon ved hjelp av loss-funksjon
5. Oppdater vektene ved å bruke back-propagation etterfulgt av gradient descent

*Mat inn data i nettverket:*

In [ ]:
i, data = next(enumerate(trainloader, 0))

In [ ]:
i

In [ ]:
inputs, labels = data

In [ ]:
len(inputs)

In [ ]:
len(labels)

In [ ]:
inputs[0]

In [ ]:
labels

In [ ]:
plt.imshow(np.transpose(torchvision.utils.make_grid(inputs[0]).numpy(), (1,2,0)))
plt.show()

In [ ]:
classes[labels[0]]

*Vi kan teste hvor godt vårt utrente nettverk med random vekter gjør det på de 10.000 testbildene. Forventer treffsikkerhet på 1/10:*

In [ ]:
check_accuracy(net)

*La oss mate bildene inn i nettverket. (Må wrappes i `Variable`.)*

In [ ]:
inputs, labels = Variable(inputs), Variable(labels)

*Forward: Finner prediksjoner på bildene.*

In [ ]:
outputs = net(inputs)

*Velg en loss-funksjon*

In [ ]:
criterion = nn.CrossEntropyLoss()

*Regn ut loss:*

In [ ]:
loss = criterion(outputs, labels)

In [ ]:
loss

*Backward: Finner alle vektenes bidrag til loss via backpropagation (automatisk derivasjon).*

In [ ]:
loss.backward()

*Oppdaterer vektene ved å bruke SGD:*

`weight = weight - learning_rate * gradient`

In [ ]:
learning_rate = 0.001

In [ ]:
optimizer = optim.SGD(net.parameters(), lr=learning_rate)

*Optimering: Bruker SGD til å modifisere alle vektene litt (med satt learning rate).*

In [ ]:
optimizer.step()

*Nå har vi trent nettet bittelitt. Tester accuracy på nytt:*

In [ ]:
check_accuracy(net)

*Gjentar prosessen med neste batch av 4 bilder:*

In [ ]:
i, data = next(enumerate(trainloader, 0))
inputs, labels = data
inputs, labels = Variable(inputs), Variable(labels)

In [ ]:
# Må nulle ut gradientene mellom hver gang, ellers akkumuleres de:
optimizer.zero_grad()

In [ ]:
outputs = net(inputs)

In [ ]:
loss = criterion(outputs, labels)

In [ ]:
loss

In [ ]:
loss.backward()

In [ ]:
optimizer.step()

In [ ]:
check_accuracy(net)

*Kan gjøre dette om og om igjen:*

In [ ]:
i, data = next(enumerate(trainloader, 0))
inputs, labels = data
inputs, labels = Variable(inputs), Variable(labels)
optimizer.zero_grad()
outputs = net(inputs)
loss = criterion(outputs, labels)
print(f"Loss: {loss}")
loss.backward()
optimizer.step()
check_accuracy(net)

*Tungvint... Krever mange manuelle gjennomkjøringer.. Hvor mange? Antall bilder:*

In [ ]:
len(trainloader.dataset)

*Antall batches på 4 hver:*

In [ ]:
len(trainloader.dataset)/4

*Må kjøre blokken over 12500 ganger for å komme én gang gjennom treningsdata.. Bedre å lage en loop:*

In [ ]:
# Antall ganger gjennom hele treningsdatasettet
num_epochs = 2

In [ ]:
# %load loop.py
for epoch in range(num_epochs): 

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = Variable(inputs), Variable(labels)
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.data[0]
        if i % 2000 == 1999:    # print hver 2000-ende batch         
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            check_accuracy(net)
            running_loss = 0.0

print('Ferdig')

<img src="assets/tommelopp.png">

# Samme nettverk i Keras (og på GPU)

<img src="assets/kerastf.png">

In [ ]:
!nvidia-smi

In [ ]:
(x_train, y_train), (x_test, y_test) = get_cifar10_keras()

In [ ]:
plt.imshow(x_train[0])
plt.show()

In [ ]:
y_train[0]

Definer nettverket:

In [ ]:
input_size = 32*32*3
hidden_size = 84

In [ ]:
model = Sequential()

model.add(Flatten(input_shape=x_train.shape[1:]))
model.add(Dense(hidden_size, activation='tanh'))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
learning_rate = 0.001

In [ ]:
opt = keras.optimizers.sgd(lr=learning_rate)

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train,
              batch_size=4,
              epochs=1,
              validation_data=(x_test, y_test))

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
model.fit(x_train, y_train,
              batch_size=4,
              epochs=4,
              validation_data=(x_test, y_test))

# Et moderne nettverk

In [ ]:
model = Sequential()


model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:], activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [ ]:
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6) # 65% etter 2 epoker

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train,
              batch_size=4,
              epochs=2,
              validation_data=(x_test, y_test))

*Forklar elementene i nettverket. Konvolusjoner, max-pooling, dropout etc*

*Vi kunne dratt inn mange triks for å booste dette. Data augmentation, ensembling, hyperparametertuning, etc. Men vi kan ikke dekke alt på 90 minutter! :-)*

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=5,  
    width_shift_range=0.1,  
    height_shift_range=0.1,  
    horizontal_flip=True  
)

model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=4), epochs=2,
                                    validation_data=(x_test, y_test))

# Et _hypermoderne_ nettverk

*DenseNet*

*Pre-trent på ImageNet*

In [ ]:
(x_train, y_train), (x_test, y_test) = get_data_dn121(allData=False)

In [ ]:
model = densenet121_model()

In [ ]:
#model.summary()

In [ ]:
model.fit(x_train, y_train, 
          batch_size=16, 
          epochs=1,
          validation_data=(x_test, y_test),
          )

*Ser lovende ut, selv med bare 3.000 treningsbilder! Men tar lang tid... Vi laster heller inn noe jeg har trent i noen timer (på en 1080Ti). Med alle 50.000 treningsbilder, 10 epoker. Ellers nøyaktig samme som over:*

In [ ]:
(x_train, y_train), (x_test, y_test) = get_data_dn121(allData=True)

In [ ]:
model = densenet121_pretrained()

In [ ]:
model.evaluate(x_test, y_test)

*Knuser altså alle resultatene vi fikk over, og gjør Cifar10 omtrent ubrukelig som benchmark!*

# Mer interessant anvendelse

Gå til `2.0-medisinske-bilder.ipynb`

Men først: shut down

<img src="assets/shutdown.png">